In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import dalex as dx
import pickle
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('heart_data.csv')

In [3]:
y = df.target.values
x = df.drop(['target'], axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2,random_state=0, stratify=y)

In [4]:
knn = pickle.load(open("Modele/knn", 'rb'))
rf = pickle.load(open("Modele/random_forest", 'rb'))

In [5]:
ss = StandardScaler()
ss.fit(x_train, y_train) 
p_knn = Pipeline(
    [
        ('standardscaler', ss),
        ('KNN', knn)
    ]
)
p_rf = Pipeline(
    [
        ('Random Forest', rf)
    ]
)
p_svc = Pipeline(
    [
        ('standardscaler', ss),
        ('SVC', SVC( C = 0.5, random_state=1, probability=True))
    ]
)
p_svc.fit(x_train, y_train)
pipelines = [p_knn, p_rf, p_svc]

In [6]:
e_knn = dx.Explainer(p_knn, x_train, y_train, label = "KNN", verbose=0)
e_svc = dx.Explainer(p_svc, x_train, y_train, label = "SVC", verbose=0)
e_rf = dx.Explainer(p_rf, x_train, y_train, label = "Random Forest", verbose=0)

In [7]:
mp_knn = e_knn.model_profile()
mp_svc = e_svc.model_profile()
mp_rf = e_rf.model_profile()

Calculating ceteris paribus: 100%|█████████████████████████████████████████████████████| 20/20 [00:57<00:00,  2.87s/it]


In [8]:
var = ['age', 'sex', 'chol', 'thalach', 'exang', 'oldpeak', 'ca','cp_ta', 'thal_n', 'thal_rd']

## PDP 

*Z podwodu wielkości pliku musiałem wrzucić wykresy jako zdjęci PDP_geom.png*

Na poniższych wykresach zajmę się klikoma zmiennymi które mają największą zmienność.  
- Pierwszą moją obserwacją jest to, że praktycznie wszystkie obserwację podążają za średnią, tzn. szare wykresy się nie przecinają to może świadczyć o małej zależności zmiennych od siebie. 
- Wzmiennej *age* występują różnice pomiędzy obserwacjami. Ogólna tendencja pokzuję skok predykcji w przedziale 50-60 lat. Jest to kontrintuicyjne, oraz w obala naszą hipotezę z EDA (,,Młodsze osoby zgłaszają się do lekarza jak mają poważne obajawy, a starsi badają się profilaktycznie"). Ten stan rzeczy potwierdza się dla osób z małymi szansami na CAD (wartość 1), ale w przypadku osób z dużymi wartościami predykcji widać jej spadek przy przekroczeniu granicy 62 lat.
- Mężczyźni zgodznie z przypuszeniami mają większe wartości predykcji.
- Duży cholesterol wpływa na predykcję ale tylko  do pewnego momentu, po którym jest mało obserwacji.
- Predykcja zmniejsza wartości gdzy zmienna *thalach* (maksymalne tętno) rośnie co jest zgodne ze zdaniem lekarza który (,,Osoby chorę na serce przerywają test siłowy zanim wystąpią duże wartości tętna")
- Po raz kolejny potwierdza się istotność zmiennych *ca* oraz *thal* ponieważ to przy nich występują największe skoki.

In [ ]:
#mp_rf.plot(variables=var, geom="profiles")

## Porówanie modeli

- Prawie dla każdej zmiennej KNN ma średnio mniejsze predykcje niż SVC.
- KNN i SVC nie wyłapały nagłej zmienności dla wieku
- Oba również znacznie bardziej zaniżają predykcje dla kobiet niż RF
- Dla zmiennej *thalach* SVC przejawia znacznie mniejszą zmienność
- SVC zadziwiająco mocno zwiększa wartość predykcji wraz ze wzrostem *ca* co jest raczej błędne ponieważ w innych wyjaśnieniach różnica pomiędzy wartościami 1-3 nie była aż tak istotna. Prawdopodobnie wynika to z tego, że model chciał dostosować prostą do dużego skoku pomiędzy wartościami 0 oraz 1.
- Warto zauważyć, że nawet jeśli wykresy różnych modeli nie pokrywają się dla zmiennyc kategorycznych to i tak są dość zgodne poniważ istotne są tylko punkty graniczne w których wartości wykresów są podobne.

In [10]:
mp_rf.plot([mp_knn, mp_svc], variables=var)

In [11]:
ale_rf = e_rf.model_profile(type = 'accumulated')
pdp_rf = e_rf.model_profile(type = 'partial')

Calculating ceteris paribus: 100%|█████████████████████████████████████████████████████| 20/20 [00:35<00:00,  1.76s/it]


## Porównanie ALE z PDP

Po porówaniu wykresów obu metod nie widać różnic w kształcie wykresów ale za to występują przesunięcia względem predykcji. Wyniki są bardzo interesujące, ponieważ umożliwia nam to zobaczenie które zmienne są mocno zależne od innych. Taką zmienną jest np. *thalach* lub *thal_n*. Przy czym dla *thal_rd* już nie ma tak znaczącej różnicy.

In [12]:
ale_rf.result['_label_'] = "ALE"
pdp_rf.result['_label_'] = "PDP"

In [13]:
ale_rf.plot(pdp_rf, variables=var)

## Wniski 

Metody PDP bardzo dobrze i rzetelnie oddają specyfikę modelu. Wyniki pokrywają się z naszymi poprzednimi założeniami jak i wnioskami z innych metod XAI. Możemy też zweryfikować wyniki poprzednej metody 

## 